# FBA-Bench v3 Interactive Analysis Example

This notebook demonstrates how to connect to a live FBA-Bench simulation for interactive data analysis using the Jupyter Connector (Observer Mode).

## Overview

The `JupyterConnector` provides a secure, read-only interface to live FBA-Bench simulations. Key features:

- **Strictly Read-Only**: Zero ability to modify simulation state
- **Real-Time Data**: WebSocket connection for live event streaming
- **Pandas Integration**: Direct DataFrame export for analysis
- **Automatic Reconnection**: Robust connection management

## Prerequisites

1. Start the FBA-Bench API server: `python api_server.py`
2. Ensure the simulation is running with some activity
3. Install required packages: `pip install -r requirements.txt`

## Step 1: Import Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import time

# Import our FBA-Bench connector
from jupyter_connector import connect_to_simulation, JupyterConnector

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## Step 2: Connect to Live Simulation

In [ ]:
# Connect to the running FBA-Bench simulation
connector = connect_to_simulation(api_url="http://localhost:8000")

# Check connection status
status = connector.get_connection_status()
print("\n📊 Connection Status:")
for key, value in status.items():
    print(f"  {key}: {value}")

## Step 3: Fetch Current Simulation State

In [ ]:
# Refresh to get latest data
connector.refresh_snapshot()

# Get simulation snapshot as DataFrame
snapshot_df = connector.get_snapshot_df()
print("📸 Current Simulation Snapshot:")
print(snapshot_df.T)  # Transpose for better readability

# Get raw snapshot for detailed inspection
snapshot_dict = connector.get_snapshot_dict()
print(f"\n🎯 Current Tick: {snapshot_dict.get('tick', 'Unknown')}")
print(f"⏰ Timestamp: {snapshot_dict.get('timestamp', 'Unknown')}")

## Step 4: Analyze Financial Performance

In [ ]:
# Extract financial metrics
if not snapshot_df.empty:
    # Get financial columns
    financial_cols = [col for col in snapshot_df.columns if 'metrics_' in col or 'cash_balance' in col]
    
    print("💰 Financial Metrics:")
    for col in financial_cols:
        value = snapshot_df[col].iloc[0]
        print(f"  {col}: ${value:.2f}")
    
    # Create financial summary
    financial_summary = snapshot_df[financial_cols].iloc[0]
    
    # Plot financial overview
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    metrics_to_plot = [col for col in financial_cols if any(metric in col for metric in ['revenue', 'profit', 'cash'])]
    
    if metrics_to_plot:
        values = [financial_summary[col] for col in metrics_to_plot]
        labels = [col.replace('metrics_', '').replace('_', ' ').title() for col in metrics_to_plot]
        
        bars = ax.bar(labels, values, alpha=0.7)
        ax.set_title('Current Financial Performance', fontsize=14, fontweight='bold')
        ax.set_ylabel('Amount ($)', fontsize=12)
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.01, 
                   f'${value:.2f}', ha='center', va='bottom', fontweight='bold')
        
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ No snapshot data available")

## Step 5: Analyze Competitor Landscape

In [ ]:
# Extract competitor data
if not snapshot_df.empty:
    competitor_cols = [col for col in snapshot_df.columns if 'competitor_' in col]
    
    if competitor_cols:
        print("🏢 Competitor Analysis:")
        
        # Extract competitor prices
        price_cols = [col for col in competitor_cols if 'price' in col]
        if price_cols:
            competitor_prices = {}
            for col in price_cols:
                # Extract competitor name from column
                comp_name = col.replace('competitor_', '').replace('_price', '')
                price = snapshot_df[col].iloc[0]
                competitor_prices[comp_name] = price
                print(f"  {comp_name}: ${price:.2f}")
            
            # Plot competitor price comparison
            if competitor_prices:
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                
                names = list(competitor_prices.keys())
                prices = list(competitor_prices.values())
                
                bars = ax.bar(names, prices, alpha=0.7, color='coral')
                ax.set_title('Competitor Price Comparison', fontsize=14, fontweight='bold')
                ax.set_ylabel('Price ($)', fontsize=12)
                ax.set_xlabel('Competitors', fontsize=12)
                
                # Add value labels
                for bar, price in zip(bars, prices):
                    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(prices)*0.01, 
                           f'${price:.2f}', ha='center', va='bottom', fontweight='bold')
                
                # Add average line
                avg_price = np.mean(prices)
                ax.axhline(y=avg_price, color='red', linestyle='--', alpha=0.7, 
                          label=f'Average: ${avg_price:.2f}')
                ax.legend()
                
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()
                
                print(f"\n📊 Market Analysis:")
                print(f"  Average Competitor Price: ${avg_price:.2f}")
                print(f"  Price Range: ${min(prices):.2f} - ${max(prices):.2f}")
                print(f"  Price Std Dev: ${np.std(prices):.2f}")
    else:
        print("  No competitor data available")
else:
    print("⚠️ No snapshot data available")

## Step 6: Real-Time Event Stream Analysis

In [ ]:
# Get recent events as DataFrame
events_df = connector.get_events_df(limit=50)

if not events_df.empty:
    print(f"📡 Recent Events ({len(events_df)} total):")
    print(events_df[['timestamp', 'event_type']].tail(10))
    
    # Analyze event frequency
    event_counts = events_df['event_type'].value_counts()
    print(f"\n📈 Event Type Distribution:")
    for event_type, count in event_counts.items():
        print(f"  {event_type}: {count}")
    
    # Plot event timeline
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Event frequency over time
    if len(events_df) > 1:
        events_df['minute'] = events_df['timestamp'].dt.floor('min')
        events_per_minute = events_df.groupby('minute').size()
        
        ax1.plot(events_per_minute.index, events_per_minute.values, marker='o', alpha=0.7)
        ax1.set_title('Event Frequency Over Time', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Events per Minute')
        ax1.grid(True, alpha=0.3)
    
    # Event type distribution pie chart
    if len(event_counts) > 1:
        ax2.pie(event_counts.values, labels=event_counts.index, autopct='%1.1f%%', 
               startangle=90, alpha=0.8)
        ax2.set_title('Event Type Distribution', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No events available in buffer")

## Step 7: Financial Transaction History

In [ ]:
# Get financial transaction history
financial_history = connector.get_financial_history_df()

if not financial_history.empty:
    print(f"💳 Financial Transaction History ({len(financial_history)} transactions):")
    print(financial_history.tail())
    
    # Analyze sales if available
    sales_events = financial_history[financial_history['event_type'] == 'SaleOccurred']
    if not sales_events.empty:
        print(f"\n🛍️ Sales Analysis:")
        print(f"  Total Sales: {len(sales_events)}")
        
        if 'sale_price' in sales_events.columns:
            total_revenue = sales_events['sale_price'].sum()
            avg_sale_price = sales_events['sale_price'].mean()
            print(f"  Total Revenue: ${total_revenue:.2f}")
            print(f"  Average Sale Price: ${avg_sale_price:.2f}")
            
            # Plot sales over time
            fig, ax = plt.subplots(1, 1, figsize=(12, 6))
            
            sales_events['hour'] = sales_events['timestamp'].dt.floor('H')
            hourly_sales = sales_events.groupby('hour')['sale_price'].agg(['count', 'sum'])
            
            ax2 = ax.twinx()
            
            line1 = ax.plot(hourly_sales.index, hourly_sales['count'], 
                           marker='o', color='blue', alpha=0.7, label='Sales Count')
            line2 = ax2.plot(hourly_sales.index, hourly_sales['sum'], 
                            marker='s', color='green', alpha=0.7, label='Revenue ($)')
            
            ax.set_xlabel('Time')
            ax.set_ylabel('Number of Sales', color='blue')
            ax2.set_ylabel('Revenue ($)', color='green')
            ax.set_title('Sales Performance Over Time', fontsize=14, fontweight='bold')
            
            # Combine legends
            lines1, labels1 = ax.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
            
            plt.tight_layout()
            plt.show()
    
    # Analyze price changes
    price_events = financial_history[financial_history['event_type'] == 'ProductPriceUpdated']
    if not price_events.empty:
        print(f"\n💰 Price Change Analysis:")
        print(f"  Price Updates: {len(price_events)}")
        
        if 'new_price' in price_events.columns:
            recent_prices = price_events['new_price'].tail(10)
            print(f"  Recent Prices: {recent_prices.tolist()}")
            
            # Plot price evolution
            fig, ax = plt.subplots(1, 1, figsize=(12, 6))
            
            ax.plot(price_events['timestamp'], price_events['new_price'], 
                   marker='o', alpha=0.7, linewidth=2)
            ax.set_title('Price Evolution Over Time', fontsize=14, fontweight='bold')
            ax.set_xlabel('Time')
            ax.set_ylabel('Price ($)')
            ax.grid(True, alpha=0.3)
            
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
else:
    print("⚠️ No financial transaction history available")

## Step 8: Live Event Monitoring (Advanced)

In [ ]:
# Monitor live events for a short period
print("🔴 Starting live event monitoring for 30 seconds...")
print("(Events will appear below as they happen)")

# Get initial event count
initial_count = len(connector.get_event_stream())

# Monitor for 30 seconds
start_time = time.time()
while time.time() - start_time < 30:
    current_count = len(connector.get_event_stream())
    
    if current_count > initial_count:
        # New events detected
        new_events = connector.get_event_stream()[initial_count:]
        for event in new_events:
            event_type = event['data'].get('event_type', 'Unknown')
            timestamp = event['timestamp'].strftime('%H:%M:%S')
            print(f"  [{timestamp}] {event_type}")
        
        initial_count = current_count
    
    time.sleep(1)  # Check every second

final_count = len(connector.get_event_stream())
new_events_detected = final_count - (initial_count if 'initial_count' in locals() else 0)
print(f"\n✅ Monitoring complete. Detected {new_events_detected} new events.")

## Step 9: Custom Analysis Functions

In [ ]:
# Define some custom analysis functions

def analyze_market_efficiency(connector):
    """Analyze how efficiently the market is operating."""
    events = connector.get_event_stream()
    
    if not events:
        return "No events available for analysis"
    
    # Count different event types
    event_types = {}
    for event in events:
        event_type = event['data'].get('event_type', 'Unknown')
        event_types[event_type] = event_types.get(event_type, 0) + 1
    
    # Calculate efficiency metrics
    total_events = len(events)
    sales_events = event_types.get('SaleOccurred', 0)
    price_events = event_types.get('ProductPriceUpdated', 0)
    
    sales_ratio = sales_events / total_events if total_events > 0 else 0
    price_change_ratio = price_events / total_events if total_events > 0 else 0
    
    return {
        'total_events': total_events,
        'sales_ratio': sales_ratio,
        'price_change_ratio': price_change_ratio,
        'efficiency_score': sales_ratio * 0.7 + (1 - price_change_ratio) * 0.3  # More sales, fewer price changes = efficient
    }

def get_profit_margin_trend(connector):
    """Calculate profit margin trends from recent data."""
    snapshot = connector.get_snapshot_dict()
    
    if not snapshot or 'metrics' not in snapshot:
        return None
    
    metrics = snapshot['metrics']
    
    # Extract revenue and profit
    revenue = metrics.get('total_revenue', {}).get('amount', 0)
    profit = metrics.get('total_profit', {}).get('amount', 0)
    
    if revenue > 0:
        margin = (profit / revenue) * 100
        return {
            'revenue': revenue,
            'profit': profit,
            'margin_percent': margin
        }
    
    return None

# Run custom analysis
print("🔬 Custom Market Analysis:")

efficiency = analyze_market_efficiency(connector)
print(f"\n📊 Market Efficiency Analysis:")
if isinstance(efficiency, dict):
    for key, value in efficiency.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.3f}")
        else:
            print(f"  {key}: {value}")
else:
    print(f"  {efficiency}")

profit_trend = get_profit_margin_trend(connector)
print(f"\n💰 Profit Margin Analysis:")
if profit_trend:
    print(f"  Revenue: ${profit_trend['revenue']:.2f}")
    print(f"  Profit: ${profit_trend['profit']:.2f}")
    print(f"  Margin: {profit_trend['margin_percent']:.2f}%")
else:
    print("  No profit data available")

## Step 10: Connection Cleanup

In [ ]:
# Clean up connection when done
print("🧹 Cleaning up connection...")

final_status = connector.get_connection_status()
print(f"Final events processed: {final_status['events_buffered']}")

connector.close()
print("✅ Connection closed successfully")

## Summary

This notebook demonstrated the key capabilities of the FBA-Bench Jupyter Connector:

1. **🔗 Connection Management**: Secure, read-only access to live simulations
2. **📊 Data Access**: Real-time snapshots and event streams as Pandas DataFrames
3. **📈 Financial Analysis**: Revenue, profit, and pricing trend analysis
4. **🏢 Competitor Monitoring**: Market landscape and competitor behavior analysis
5. **🔴 Live Event Tracking**: Real-time monitoring of simulation events
6. **🔬 Custom Analytics**: Extensible framework for advanced analysis

## Next Steps

- **Time Series Analysis**: Build historical datasets for trend analysis
- **Machine Learning**: Train models on competitor behavior patterns
- **Advanced Visualization**: Create interactive dashboards with Plotly
- **Automated Reporting**: Schedule periodic analysis reports
- **Multi-Simulation Comparison**: Compare different simulation runs

The FBA-Bench v3 platform now provides world-class research capabilities for e-commerce simulation analysis!